# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [ ]:
# imports

from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment

load_dotenv()
openai = OpenAI()


In [ ]:
# Aquí está la pregunta; escribe sobre esto para preguntar algo nuevo

question = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# prompts

system_prompt = "Eres un tutor técnico útil que responde preguntas sobre código Python, ingeniería de software, ciencia de datos y LLM"
user_prompt = "Por favor, da una explicación detallada de la siguiente pregunta: " + question

In [ ]:
# messages

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
# Haz que gpt-4o-mini responda, con streaming

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Haz que Llama 3.2 responda

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

# ¡Felicitaciones!

Puedes mejorarlo si incluyes la pregunta usando
`my_question = input("Por favor, introduce tu pregunta:")`

Y luego creas las indicaciones y realizas las llamadas de manera interactiva.